In [ ]:
!pip install groq
import groq

print(groq.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 5.7 MB/s eta 0:00:00
1.0.0


In [ ]:
#model baglanti testi
from google.colab import userdata
groq_api_key =  userdata.get('GROQ_API_KEY')

from groq import Groq

client = Groq(api_key=groq_api_key)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello' how are you?",
        }
    ],
    model="llama-3.3-70b-versatile" # "llama-3.1-70b-versatile", # llama3-70b-8192
)

print(chat_completion.choices[0].message.content)

Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [ ]:
import re
import math
import json
from collections import Counter
from google.colab import files

In [ ]:
# 2. RAG MOTORU (In-Memory Vector Store)
# ------------------------------------------------------------------


class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text):
        chunks = [s.strip() for s in text.split('.') if s.strip()]
        for chunk in chunks:
            self.vector_store.append({
                "content": chunk,
                "vector": self.get_embedding(chunk)
            })

    def retrieve(self, query, top_k=2):
        query_vec = self.get_embedding(query)
        scores = []
        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"]))
        scores.sort(key=lambda x: x[0], reverse=True)
        return "\n".join([item[1] for item in scores[:top_k] if item[0] > 0])

# RAG Data Ingestion


# JSON dosyasını yükle
print(" JSON dosyasını yükleyin (gtip_codes.json):")
uploaded = files.upload()

gtip_data = None
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        gtip_data = json.load(f)
    print(f" {filename} yüklendi! {len(gtip_data['products'])} ürün bulundu.")

# PDF'leri yükle
print("\nPDF dosyalarını yükleyin ")
uploaded_pdfs = files.upload()
uploaded_pdfs = files.upload()
uploaded_pdfs = files.upload()


print("\n Veri yükleme tamamlandı!")

 JSON dosyasını yükleyin :


Saving gtip_codes.json to gtip_codes (3).json
 gtip_codes (3).json yüklendi! 200 ürün bulundu.

PDF dosyalarını yükleyin 


Saving 7.5.13472_gümrük_yönetmeliği.pdf to 7.5.13472_gümrük_yönetmeliği (2).pdf


Saving 1.5.3065_kdv_kanunu.pdf to 1.5.3065_kdv_kanunu.pdf


Saving 1.5.4458_gümrük_kararı.pdf to 1.5.4458_gümrük_kararı.pdf

 Veri yükleme tamamlandı!


In [ ]:
# PDF PROCESSING (RAG ICIN)

# PDF'leri RAG'e yukle
rag_engine = SimpleRAG()

if 'uploaded_pdfs' in locals() and uploaded_pdfs:
    print("\nPDF'ler isleniyor...")

    try:
        import PyPDF2
    except ImportError:
        !pip install PyPDF2
        import PyPDF2

    for filename in uploaded_pdfs.keys():
        print(f"   {filename} okunuyor...")

        # PDF'i oku
        with open(filename, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()

        # RAG'e ekle
        rag_engine.ingest(text)
        print(f"   {filename} RAG'e eklendi!")

    print(f"\nToplam {len(rag_engine.vector_store)} chunk RAG'de!")
else:
    print("PDF yuklenmedi, RAG bos!")


# ARACLAR (TOOLS) - GUMRUK PROJESI

def search_gtip_database(query: str) -> str:
    """
    GTiP veritabaninda urun ara. Urun adi ile GTiP kodu, gumruk vergisi ve KDV oranlarini bul.

    Args:
        query: Aranacak urun adi (orn: 'laptop', 'kahve', 'ayakkabi')

    Returns:
        Bulunan urun bilgileri (GTiP kodu, vergi oranlari)
    """
    print(f"\n[GTiP SEARCH] Aranan: '{query}'")

    results = []
    query_lower = query.lower()

    for product in gtip_data['products']:
        if query_lower in product['name'].lower():
            results.append(product)

    if not results:
        return f"'{query}' icin sonuc bulunamadi."

    # Ilk 3 sonucu dondur
    output = f"'{query}' icin bulunan urunler:\n\n"
    for i, prod in enumerate(results[:3], 1):
        output += f"{i}. {prod['name']}\n"
        output += f"   GTiP: {prod['gtip']}\n"
        output += f"   Kategori: {prod['category']}\n"
        output += f"   Gumruk Vergisi: %{prod['customs_duty']}\n"
        output += f"   KDV: %{prod['vat']}\n"
        output += f"   Aciklama: {prod['gtip_description']}\n\n"

    return output


def calculate_customs_duty(params: str) -> str:
    """
    Gumruk vergisi, KDV ve toplam ithalat maliyetini hesapla.

    Args:
        params: "cif_value, duty_rate, vat_rate" formatinda (orn: "1360000, 0, 20")

    Returns:
        Detayli vergi hesaplamasi
    """
    parts = [p.strip() for p in params.split(',')]
    cif_value = float(parts[0])
    duty_rate = float(parts[1])
    vat_rate = float(parts[2])

    print(f"\n[TAX CALC] CIF: {cif_value} TL, Gumruk: %{duty_rate}, KDV: %{vat_rate}")

    customs_duty = cif_value * (duty_rate / 100)
    tax_base = cif_value + customs_duty
    vat = tax_base * (vat_rate / 100)
    total_cost = cif_value + customs_duty + vat

    result = f"""
GUMRUK VERGISI HESAPLAMASI:
CIF Degeri:           {cif_value:,.2f} TL
Gumruk Vergisi (%{duty_rate}): {customs_duty:,.2f} TL
Vergi Matrahi:        {tax_base:,.2f} TL
KDV (%{vat_rate}):              {vat:,.2f} TL
TOPLAM MALIYET:       {total_cost:,.2f} TL
"""
    return result


def calculate_shipping_cost(params: str) -> str:
    """
    Navlun (kargo) maliyetini hesapla.

    Args:
        params: "weight_kg, method" formatinda (orn: "10, hava")

    Returns:
        Navlun maliyeti
    """
    parts = [p.strip() for p in params.split(',')]
    weight_kg = float(parts[0])
    method = parts[1]

    print(f"\n[SHIPPING] {weight_kg} kg, Yontem: {method}")

    rates = {
        "hava": 45.0,
        "deniz": 8.0,
        "kara": 15.0
    }

    method_lower = method.lower()

    if method_lower not in rates:
        return f"Hata: '{method}' gecersiz. Secenekler: hava, deniz, kara"

    cost = weight_kg * rates[method_lower]

    result = f"""
NAVLUN MALIYETI:
Tasima Yontemi: {method.upper()}
Agirlik:        {weight_kg} kg
Birim Fiyat:    {rates[method_lower]} TL/kg
TOPLAM NAVLUN:  {cost:,.2f} TL
"""
    return result


def calculator(expression: str) -> str:
    """
    Matematiksel islem yap.

    Args:
        expression: Matematiksel ifade (orn: '100 * 1.2 + 50')

    Returns:
        Hesaplama sonucu
    """
    print(f"\n[CALC] Islem: {expression}")

    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Hata: Gecersiz karakter."

        result = eval(expression)
        return f"{expression} = {result:,.2f}"
    except Exception as e:
        return f"Hesaplama hatasi: {str(e)}"


def search_knowledge_base(query: str) -> str:
    """
    Gumruk mevzuatinda bilgi ara (PDF'lerden).

    Args:
        query: Aranacak konu (orn: 'gumruk vergisi muafiyeti')

    Returns:
        Bulunan mevzuat bilgileri
    """
    print(f"\n[RAG SEARCH] Mevzuat aramasi: '{query}'")

    result = rag_engine.retrieve(query, top_k=3)

    if not result:
        return "Mevzuatta bu konuyla ilgili bilgi bulunamadi."

    return f"Mevzuat Bilgisi:\n{result}"


# Tool listesi
tools = [
    search_gtip_database,
    calculate_customs_duty,
    calculate_shipping_cost,
    calculator,
    search_knowledge_base
]

# Tool'lari yazdir
tools_with_desc = "\n\n".join([f"{tool.__name__}:\n{tool.__doc__}" for tool in tools])
print("\n5 TOOL TANIMLANDI:")
print(tools_with_desc)

# Tool dictionary
known_actions = {tool.__name__: tool for tool in tools}

print("\nTool'lar hazir!")


PDF'ler isleniyor...
   1.5.4458_gümrük_kararı.pdf okunuyor...
   1.5.4458_gümrük_kararı.pdf RAG'e eklendi!

Toplam 2098 chunk RAG'de!

5 TOOL TANIMLANDI:
search_gtip_database:

    GTiP veritabaninda urun ara. Urun adi ile GTiP kodu, gumruk vergisi ve KDV oranlarini bul.

    Args:
        query: Aranacak urun adi (orn: 'laptop', 'kahve', 'ayakkabi')

    Returns:
        Bulunan urun bilgileri (GTiP kodu, vergi oranlari)
    

calculate_customs_duty:

    Gumruk vergisi, KDV ve toplam ithalat maliyetini hesapla.
    
    Args:
        params: "cif_value, duty_rate, vat_rate" formatinda (orn: "1360000, 0, 20")
    
    Returns:
        Detayli vergi hesaplamasi
    

calculate_shipping_cost:

    Navlun (kargo) maliyetini hesapla.
    
    Args:
        params: "weight_kg, method" formatinda (orn: "10, hava")
    
    Returns:
        Navlun maliyeti
    

calculator:

    Matematiksel islem yap.

    Args:
        expression: Matematiksel ifade (orn: '100 * 1.2 + 50')

    Returns:


In [ ]:
# AGENT VE PROMPT YAPISI

system_prompt = f"""
Sen Turkiye Gumruk ve Dis Ticaret Danismanisin.

GOREVLER:
1. GTiP kodlari ve gumruk vergisi oranlarini bul
2. Gumruk vergisi, KDV ve toplam maliyeti hesapla
3. Navlun (kargo) maliyetlerini hesapla
4. Ithalat maliyeti karsilastirmalari yap
5. Gumruk mevzuati hakkinda bilgi ver

REACT DONGUSU:
Her adimda su sekilde calis:
- Thought: Ne yapmam gerektigini dusun
- Action: Bir tool cagir
- PAUSE: Dur ve sonucu bekle
- Observation: Tool sonucunu gor
- Devam et veya Answer ver

TOOL KULLANIM KURALLARI:
1. GTiP kodu aramak icin: search_gtip_database
2. Vergi hesaplamak icin: calculate_customs_duty
3. Navlun hesaplamak icin: calculate_shipping_cost
4. Matematik islem icin: calculator (KAFANDAN HESAPLAMA!)
5. Mevzuat aramak icin: search_knowledge_base

MEVCUT TOOLS:
{tools_with_desc}

ORNEK SENARYO 1 (Single-Hop):
Question: Laptop icin GTiP kodu ve gumruk vergisi nedir?
Thought: Once GTiP veritabaninda laptop arayayim.
Action: search_gtip_database: laptop
PAUSE
Observation: Laptop (Dizustu Bilgisayar), GTiP: 8471.30.00, Gumruk Vergisi: %0, KDV: %20
Answer: Laptop icin GTiP kodu 8471.30.00'dir. Gumruk vergisi %0, KDV %20'dir.

ORNEK SENARYO 2 (Multi-Hop):
Question: 50 iPhone ithal etmek istiyorum. Toplam maliyet ne kadar? (Birim fiyat: 800 USD, kur: 34 TL, agirlik: 10 kg, hava kargo)
Thought: Oncelikle akilli telefon icin GTiP kodunu ve vergi oranlarini bulmaliyim.
Action: search_gtip_database: akilli telefon
PAUSE
Observation: Akilli Telefon, GTiP: 8517.12.00, Gumruk Vergisi: %0, KDV: %20
Thought: Simdi CIF degerini hesaplamaliyim: 50 * 800 * 34
Action: calculator: 50 * 800 * 34
PAUSE
Observation: 1360000.0
Thought: CIF = 1,360,000 TL. Simdi gumruk vergisi ve KDV hesaplayalim (gumruk %0, KDV %20)
Action: calculate_customs_duty: 1360000, 0, 20
PAUSE
Observation: CIF: 1,360,000 TL, Gumruk: 0 TL, KDV: 272,000 TL, TOPLAM: 1,632,000 TL
Thought: Simdi navlun maliyetini hesaplayalim (10 kg, hava)
Action: calculate_shipping_cost: 10, hava
PAUSE
Observation: Navlun: 450 TL
Thought: Toplam maliyet = Mal + Vergi + Navlun
Action: calculator: 1632000 + 450
PAUSE
Observation: 1632450.0
Answer: 50 iPhone ithalatinin toplam maliyeti 1,632,450 TL'dir. (Mal: 1,360,000 TL + Vergi: 272,000 TL + Navlun: 450 TL)

ORNEK SENARYO 3 (Comparison):
Question: 1000 ayakkabi ithal edecegim (100 kg). Deniz mi hava mi daha ekonomik?
Thought: Once her iki yontem icin navlun hesaplayalim.
Action: calculate_shipping_cost: 100, deniz
PAUSE
Observation: Navlun: 800 TL
Thought: Simdi hava kargo maliyetini hesaplayalim.
Action: calculate_shipping_cost: 100, hava
PAUSE
Observation: Navlun: 4,500 TL
Thought: Deniz yolu cok daha ucuz.
Answer: Deniz yolu daha ekonomiktir. Deniz: 800 TL, Hava: 4,500 TL. Fark: 3,700 TL tasarruf.

ONEMLI:
- Her hesaplamada calculator kullan!
- Turkcede cevap ver
- Adim adim dusun
- Tool sonuclarini kullan
""".strip()


In [ ]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # stop parametresine Llama-3'ün özel token başlangıcını ekliyoruz
        result = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            # PAUSE ve Observation'a ek olarak '<|' karakterini de durdurucu olarak ekliyoruz
            stop=["PAUSE", "Observation:", "<|"]
        )
        return result.choices[0].message.content

In [ ]:
import re
import math
import json
from collections import Counter
from google.colab import files, userdata
from groq import Groq

# API AYARLARI
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)
except Exception as e:
    print("Error: GROQ_API_KEY not found. Please check Colab Secrets.")

# RAG MOTORU
class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text):
        chunks = [p.strip() for p in text.split('\n\n') if p.strip() and len(p.strip()) > 50]
        for chunk in chunks:
            if len(chunk) > 1000:
                sentences = [s.strip() for s in chunk.split('.') if s.strip()]
                for sent in sentences:
                    if len(sent) > 20:
                        self.vector_store.append({
                            "content": sent,
                            "vector": self.get_embedding(sent)
                        })
            else:
                self.vector_store.append({
                    "content": chunk,
                    "vector": self.get_embedding(chunk)
                })

    def retrieve(self, query, top_k=2):
        query_vec = self.get_embedding(query)
        scores = []
        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"]))
        scores.sort(key=lambda x: x[0], reverse=True)
        return "\n".join([item[1] for item in scores[:top_k] if item[0] > 0])

# VERI YUKLEME
print("JSON dosyasini yukleyin:")
uploaded = files.upload()

gtip_data = None
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        gtip_data = json.load(f)
    print(f"{filename} yuklendi! {len(gtip_data['products'])} urun bulundu.")

print("\nPDF dosyalarini yukleyin:")
uploaded_pdfs = files.upload()

rag_engine = SimpleRAG()

if uploaded_pdfs:
    print("PDF'ler isleniyor...")
    try:
        import PyPDF2
    except ImportError:
        get_ipython().system('pip install PyPDF2')
        import PyPDF2

    for filename in uploaded_pdfs.keys():
        print(f"   {filename} okunuyor...")
        with open(filename, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
        rag_engine.ingest(text)
        print(f"   {filename} RAG'e eklendi!")
    print(f"Toplam {len(rag_engine.vector_store)} chunk RAG'de!")

# TOOLS
def search_gtip_database(query: str) -> str:
    """GTiP veritabaninda urun ara."""
    print(f"\n[GTiP SEARCH] Aranan: '{query}'")

    # Turkce karakter normalizasyonu
    def normalize_turkish(text):
        replacements = {
            'ı': 'i', 'İ': 'i',
            'ğ': 'g', 'Ğ': 'g',
            'ü': 'u', 'Ü': 'u',
            'ş': 's', 'Ş': 's',
            'ö': 'o', 'Ö': 'o',
            'ç': 'c', 'Ç': 'c'
        }
        for tr_char, en_char in replacements.items():
            text = text.replace(tr_char, en_char)
        return text.lower()

    results = []
    query_normalized = normalize_turkish(query)

    for product in gtip_data['products']:
        product_normalized = normalize_turkish(product['name'])
        if query_normalized in product_normalized:
            results.append(product)

    if not results:
        return f"'{query}' icin sonuc bulunamadi."

    output = f"'{query}' icin bulunan urunler:\n\n"
    for i, prod in enumerate(results[:3], 1):
        output += f"{i}. {prod['name']}\n"
        output += f"   GTiP: {prod['gtip']}\n"
        output += f"   Gumruk Vergisi: %{prod['customs_duty']}\n"
        output += f"   KDV: %{prod['vat']}\n\n"
    return output

def calculate_customs_duty(params: str) -> str:
    """Gumruk vergisi hesapla. Format: cif_value, duty_rate, vat_rate"""
    parts = [p.strip() for p in params.split(',')]
    cif_value = float(parts[0])
    duty_rate = float(parts[1])
    vat_rate = float(parts[2])
    print(f"\n[TAX CALC] CIF: {cif_value} TL, Gumruk: %{duty_rate}, KDV: %{vat_rate}")
    customs_duty = cif_value * (duty_rate / 100)
    tax_base = cif_value + customs_duty
    vat = tax_base * (vat_rate / 100)
    total_cost = cif_value + customs_duty + vat
    return f"CIF: {cif_value:,.2f} TL, Gumruk: {customs_duty:,.2f} TL, KDV: {vat:,.2f} TL, TOPLAM: {total_cost:,.2f} TL"

def calculate_shipping_cost(params: str) -> str:
    """Navlun hesapla. Format: weight_kg, method"""
    parts = [p.strip() for p in params.split(',')]
    weight_kg = float(parts[0])
    method = parts[1].lower()
    print(f"\n[SHIPPING] {weight_kg} kg, Yontem: {method}")
    rates = {"hava": 45.0, "deniz": 8.0, "kara": 15.0}
    if method not in rates:
        return f"Hata: '{method}' gecersiz."
    cost = weight_kg * rates[method]
    return f"{method.upper()}: {cost:,.2f} TL"

def calculator(expression: str) -> str:
    """Matematik islem."""
    print(f"\n[CALC] Islem: {expression}")
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Hata: Gecersiz karakter."
        result = eval(expression)
        return f"{result:,.2f}"
    except Exception as e:
        return f"Hesaplama hatasi: {str(e)}"

def search_knowledge_base(query: str) -> str:
    """Gumruk mevzuatinda ara."""
    print(f"\n[RAG SEARCH] Mevzuat aramasi: '{query}'")
    result = rag_engine.retrieve(query, top_k=10)
    if not result:
        return "Mevzuatta bilgi bulunamadi."
    return f"Mevzuat: {result}"

tools = [search_gtip_database, calculate_customs_duty, calculate_shipping_cost, calculator, search_knowledge_base]
tools_with_desc = "\n\n".join([tool.__name__ + ": " + tool.__doc__ for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])


system_prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

GUIDELINES:
1. Use 'search_gtip_database' for product GTiP codes.
2. Use 'calculate_customs_duty' for tax calculations.
3. Use 'calculator' for ANY math. DO NOT calculate in your head.
4. Use 'search_knowledge_base' for customs law questions.
5. ALWAYS end with "Answer: [your answer]"

IMPORTANT RULES:
1. When you have all information needed, immediately output Answer.
2. Do NOT search for additional confirmation if you already have the data.
3. After finding GTiP code and tax rates, go directly to Answer.

Your available actions are:
{tools_with_desc}

Example session:
Question: 50 iPhone, 800 USD each, rate 34, 10kg air cargo, total cost?
Thought: First find iPhone GTiP code
Action: search_gtip_database: iPhone
PAUSE
Observation: GTiP: 8517.12.00, Duty: 0%, VAT: 20%
Thought: Calculate CIF: 50 * 800 * 34
Action: calculator: 50 * 800 * 34
PAUSE
Observation: 1360000
Thought: Calculate taxes
Action: calculate_customs_duty: 1360000, 0, 20
PAUSE
Observation: TOTAL: 1632000 TL
Thought: Calculate shipping
Action: calculate_shipping_cost: 10, hava
PAUSE
Observation: 450 TL
Thought: Calculate total
Action: calculator: 1632000 + 450
PAUSE
Observation: 1632450
Answer: Total cost is 1,632,450 TL.

IMPORTANT: Always conclude with "Answer: [final answer]"
""".strip()

# AGENT
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        result = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            stop=["PAUSE", "Observation:", "<|"]
        )
        return result.choices[0].message.content

# QUERY LOOP
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=10):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    print(f"\nQUESTION: {question}\n")
    while i < max_turns:
        i += 1
        result = react_agent(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}")
            print(f"\n-- running {action} {action_input}")
            try:
                observation = known_actions[action](action_input)
            except Exception as e:
                observation = f"Error: {e}"
            print(f"Observation: {observation}\n")
            next_prompt = f"Observation: {observation}"
        else:
            if "Answer:" in result:
                print("\nFINAL ANSWER:")
                print(result.split("Answer:")[-1].strip())
            return



JSON dosyasini yukleyin:


Saving gtip_codes.json to gtip_codes (4).json
gtip_codes (4).json yuklendi! 200 urun bulundu.

PDF dosyalarini yukleyin:


Saving Genel Hükümler_merged.pdf to Genel Hükümler_merged (1).pdf
PDF'ler isleniyor...
   Genel Hükümler_merged (1).pdf okunuyor...
   Genel Hükümler_merged (1).pdf RAG'e eklendi!
Toplam 578 chunk RAG'de!


In [ ]:
import re
import math
import json
from collections import Counter
from google.colab import files, userdata
from groq import Groq

# API AYARLARI
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)
except Exception as e:
    print("Error: GROQ_API_KEY not found. Please check Colab Secrets.")

# RAG MOTORU
class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text):
        """
        Metni chunk'lara ayırır.
        Eğer metin "Madde X:" formatındaysa madde bazlı,
        değilse paragraf bazlı chunking yapar.
        """
        chunks = []

        if "Madde " in text and text.count("Madde ") > 5:
            print("   → Madde bazlı chunking kullanılıyor...")
            current_chunk = ""

            for line in text.split('\n'):
                if re.match(r'^Madde \d+:', line.strip()) and current_chunk:
                    if len(current_chunk.strip()) > 50:
                        chunks.append(current_chunk.strip())
                    current_chunk = line + "\n"
                else:
                    current_chunk += line + "\n"

            if current_chunk.strip() and len(current_chunk.strip()) > 50:
                chunks.append(current_chunk.strip())

        else:
            print("   → Paragraf bazlı chunking kullanılıyor...")
            chunks = [p.strip() for p in text.split('\n\n')
                     if p.strip() and len(p.strip()) > 50]

            # Çok uzun chunk'ları cümlelere böl
            final_chunks = []
            for chunk in chunks:
                if len(chunk) > 1000:
                    sentences = [s.strip() for s in chunk.split('.') if s.strip()]
                    for sent in sentences:
                        if len(sent) > 20:
                            final_chunks.append(sent)
                else:
                    final_chunks.append(chunk)
            chunks = final_chunks

        # Chunk'ları vector store'a ekle
        for chunk in chunks:
            self.vector_store.append({
                "content": chunk,
                "vector": self.get_embedding(chunk)
            })

    def retrieve(self, query, top_k=10):
        query_vec = self.get_embedding(query)
        scores = []
        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"]))
        scores.sort(key=lambda x: x[0], reverse=True)
        return "\n\n".join([item[1] for item in scores[:top_k] if item[0] > 0])

# VERI YUKLEME
print("JSON dosyasini yukleyin:")
uploaded = files.upload()

gtip_data = None
for filename in uploaded.keys():
    with open(filename, 'r', encoding='utf-8') as f:
        gtip_data = json.load(f)
    print(f"{filename} yuklendi! {len(gtip_data['products'])} urun bulundu.")

print("\nTXT/PDF dosyalarini yukleyin:")
uploaded_files = files.upload()

rag_engine = SimpleRAG()

if uploaded_files:
    print("Dosyalar isleniyor...")

    for filename in uploaded_files.keys():
        print(f"   {filename} okunuyor...")

        if filename.endswith('.txt'):
            with open(filename, 'r', encoding='utf-8') as f:
                text = f.read()
            rag_engine.ingest(text)
            print(f"   {filename} RAG'e eklendi!")

        # PDF dosyası mı?
        elif filename.endswith('.pdf'):
            try:
                import PyPDF2
            except ImportError:
                get_ipython().system('pip install PyPDF2')
                import PyPDF2

            with open(filename, 'rb') as pdf_file:
                pdf_reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text()
            rag_engine.ingest(text)
            print(f"   {filename} RAG'e eklendi!")

    print(f"\nToplam {len(rag_engine.vector_store)} chunk RAG'de!")

# TOOLS
def search_gtip_database(query: str) -> str:
    """GTiP veritabaninda urun ara."""
    print(f"\n[GTiP SEARCH] Aranan: '{query}'")

    # Turkce karakter normalizasyonu
    def normalize_turkish(text):
        replacements = {
            'ı': 'i', 'İ': 'i',
            'ğ': 'g', 'Ğ': 'g',
            'ü': 'u', 'Ü': 'u',
            'ş': 's', 'Ş': 's',
            'ö': 'o', 'Ö': 'o',
            'ç': 'c', 'Ç': 'c'
        }
        for tr_char, en_char in replacements.items():
            text = text.replace(tr_char, en_char)
        return text.lower()

    results = []
    query_normalized = normalize_turkish(query)

    for product in gtip_data['products']:
        product_normalized = normalize_turkish(product['name'])
        if query_normalized in product_normalized:
            results.append(product)

    if not results:
        return f"'{query}' icin sonuc bulunamadi."

    output = f"'{query}' icin bulunan urunler:\n\n"
    for i, prod in enumerate(results[:3], 1):
        output += f"{i}. {prod['name']}\n"
        output += f"   GTiP: {prod['gtip']}\n"
        output += f"   Gumruk Vergisi: %{prod['customs_duty']}\n"
        output += f"   KDV: %{prod['vat']}\n\n"
    return output

def calculate_customs_duty(params: str) -> str:
    """Gumruk vergisi hesapla. Format: cif_value, duty_rate, vat_rate"""
    parts = [p.strip() for p in params.split(',')]
    cif_value = float(parts[0])
    duty_rate = float(parts[1])
    vat_rate = float(parts[2])
    print(f"\n[TAX CALC] CIF: {cif_value} TL, Gumruk: %{duty_rate}, KDV: %{vat_rate}")
    customs_duty = cif_value * (duty_rate / 100)
    tax_base = cif_value + customs_duty
    vat = tax_base * (vat_rate / 100)
    total_cost = cif_value + customs_duty + vat
    return f"CIF: {cif_value:,.2f} TL, Gumruk: {customs_duty:,.2f} TL, KDV: {vat:,.2f} TL, TOPLAM: {total_cost:,.2f} TL"

def calculate_shipping_cost(params: str) -> str:
    """Navlun hesapla. Format: weight_kg, method"""
    parts = [p.strip() for p in params.split(',')]
    weight_kg = float(parts[0])
    method = parts[1].lower()
    print(f"\n[SHIPPING] {weight_kg} kg, Yontem: {method}")
    rates = {"hava": 45.0, "deniz": 8.0, "kara": 15.0}
    if method not in rates:
        return f"Hata: '{method}' gecersiz."
    cost = weight_kg * rates[method]
    return f"{method.upper()}: {cost:,.2f} TL"

def calculator(expression: str) -> str:
    """Matematik islem."""
    print(f"\n[CALC] Islem: {expression}")
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Hata: Gecersiz karakter."
        result = eval(expression)
        return f"{result:,.2f}"
    except Exception as e:
        return f"Hesaplama hatasi: {str(e)}"

def search_knowledge_base(query: str) -> str:
    """Gumruk mevzuatinda ara."""
    print(f"\n[RAG SEARCH] Mevzuat aramasi: '{query}'")
    result = rag_engine.retrieve(query, top_k=10)
    if not result:
        return "Mevzuatta bilgi bulunamadi."
    return f"Mevzuat:\n{result}"

tools = [search_gtip_database, calculate_customs_duty, calculate_shipping_cost, calculator, search_knowledge_base]
tools_with_desc = "\n\n".join([tool.__name__ + ": " + tool.__doc__ for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])


system_prompt = f"""
Sen bir ReAct (Reasoning + Acting) döngüsünde çalışan bir ajansın.
Döngü: Düşünce (Thought), Eylem (Action), DURAKLAT (PAUSE), Gözlem (Observation).
Döngünün sonunda bir Cevap (Answer) verirsin.

KULLANIM KILAVUZU:
1. Ürün GTİP kodları için 'search_gtip_database' kullan.
2. Vergi hesaplamaları için 'calculate_customs_duty' kullan.
3. HER türlü matematik işlem için 'calculator' kullan. Kafandan hesaplama YAPMA.
4. Gümrük mevzuatı soruları için 'search_knowledge_base' kullan.
5. HER ZAMAN "Cevap:" ile bitir.

KRİTİK KURALLAR:
1. Her Observation'dan sonra MUTLAKA şunlardan BİRİNİ yap:
   a) Yeterli bilgin varsa "Cevap: [cevabın]" yaz
   b) Daha fazla bilgiye ihtiyacın varsa "Action: araç_adı: girdi" yaz
2. ASLA sadece "Thought:" ile bitirme - MUTLAKA "Cevap:" veya "Action:" ekle
3. ASLA "Action: None" yazma - bu GEÇERSİZ!
4. KISMEN bilgi bulsan bile "Cevap:" yaz ve bulduğunu söyle
5. Mükemmel bilgi bekleme - eldeki verilerle cevapla
6. "Cevap:" konuşmayı bitirmenin TEK geçerli yolu

Kullanabileceğin araçlar:
{tools_with_desc}

Örnek Oturum 1 - Tam Cevap:
Soru: 50 iPhone, tanesi 800 USD, kur 34, 10kg hava kargo, toplam maliyet?
Thought: Önce iPhone GTİP kodunu bulalım
Action: search_gtip_database: iPhone
PAUSE
Observation: GTiP: 8517.12.00, Gümrük: %0, KDV: %20
Thought: CIF hesaplayalım: 50 * 800 * 34
Action: calculator: 50 * 800 * 34
PAUSE
Observation: 1360000
Thought: Vergileri hesaplayalım
Action: calculate_customs_duty: 1360000, 0, 20
PAUSE
Observation: TOPLAM: 1632000 TL
Thought: Navlun hesaplayalım
Action: calculate_shipping_cost: 10, hava
PAUSE
Observation: HAVA: 450 TL
Thought: Toplam hesaplayalım
Action: calculator: 1632000 + 450
PAUSE
Observation: 1632450
Thought: Tüm bilgilere sahibim.
Cevap: Toplam maliyet 1,632,450 TL'dir (CIF 1,360,000 TL, vergiler 272,000 TL, navlun 450 TL dahil).

Örnek Oturum 2 - Kısmi Cevap Kabul Edilir:
Soru: Gümrük beyannamesi nedir ve hangi belgeleri içerir?
Thought: Gümrük beyannamesi tanımını arayalım
Action: search_knowledge_base: gümrük beyannamesi
PAUSE
Observation: Madde 9: Gümrük beyanı, ürünlerin niteliği, değeri, alıcısı ve gönderinin nihai varış yeri gibi bilgileri içeren bir belgedir.
Thought: Tanımı buldum. Tam belge listesini bulamadım ama bulduklarımla cevap verebilirim.
Cevap: Gümrük beyannamesi, ürünlerin niteliği, değeri, alıcısı ve nihai varış yeri bilgilerini içeren bir belgedir. İthalat ve ihracat için gerekli bir beyandır.

HATIRLA:
- Observation'dan sonra hemen karar ver: Cevap mı Action mı?
- İLGİLİ bilgi bulduysan "Cevap:" yaz
- ASLA sadece "Thought:" ile bitirme - mutlaka "Cevap:" veya "Action:" ekle
- Kısmi cevaplar kabul edilir - mükemmel bilgi bekleme
- Tüm cevapları TÜRKÇE yaz
""".strip()

# AGENT
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        result = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            stop=["PAUSE", "Observation:", "<|"]
        )
        return result.choices[0].message.content

# QUERY LOOP
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=10):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    print(f"\nQUESTION: {question}\n")
    while i < max_turns:
        i += 1
        result = react_agent(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}")
            print(f"\n-- running {action} {action_input}")
            try:
                observation = known_actions[action](action_input)
            except Exception as e:
                observation = f"Error: {e}"
            print(f"Observation: {observation}\n")
            next_prompt = f"Observation: {observation}"
        else:
            if "Answer:" in result:
                print("\nFINAL ANSWER:")
                print(result.split("Answer:")[-1].strip())
            return

JSON dosyasini yukleyin:


Saving gtip_codes.json to gtip_codes.json
gtip_codes.json yuklendi! 200 urun bulundu.

TXT/PDF dosyalarini yukleyin:


Saving final_genel_TÜRK GÜMRÜK VE DIŞ TİCARET.pdf to final_genel_TÜRK GÜMRÜK VE DIŞ TİCARET.pdf
Dosyalar isleniyor...
   final_genel_TÜRK GÜMRÜK VE DIŞ TİCARET.pdf okunuyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
   → Madde bazlı chunking kullanılıyor...
   final_genel_TÜRK GÜMRÜK VE DIŞ TİCARET.pdf RAG'e eklendi!

Toplam 79 chunk RAG'de!


In [ ]:
query("Laptop icin GTiP kodu ve gumruk vergisi nedir?")


QUESTION: Laptop icin GTiP kodu ve gumruk vergisi nedir?

Thought: First find Laptop GTiP code
Action: search_gtip_database: Laptop


-- running search_gtip_database Laptop

[GTiP SEARCH] Aranan: 'Laptop'
Observation: 'Laptop' icin bulunan urunler:

1. Laptop (Dizüstü Bilgisayar)
   GTiP: 8471.30.00
   Gumruk Vergisi: %0
   KDV: %20

2. Güç Kaynağı (Laptop Şarj)
   GTiP: 8504.40.00
   Gumruk Vergisi: %0
   KDV: %20



Thought: We have found the GTiP code and tax rates for Laptop, which is 8471.30.00 with 0% customs duty and 20% VAT.
Action: No further actions needed, we can conclude.
Answer: GTiP kodu: 8471.30.00, Gumruk Vergisi: %0, KDV: %20

FINAL ANSWER:
GTiP kodu: 8471.30.00, Gumruk Vergisi: %0, KDV: %20


In [ ]:
query("50 akilli telefon ithal etmek istiyorum. Birim fiyat 800 USD, kur 34 TL, toplam agirlik 10 kg, hava kargo. Toplam maliyet ne kadar?")


QUESTION: 50 akilli telefon ithal etmek istiyorum. Birim fiyat 800 USD, kur 34 TL, toplam agirlik 10 kg, hava kargo. Toplam maliyet ne kadar?

Thought: Öncelikle akilli telefonun GTiP kodunu bulalım
Action: search_gtip_database: akilli telefon


-- running search_gtip_database akilli telefon

[GTiP SEARCH] Aranan: 'akilli telefon'
Observation: 'akilli telefon' icin bulunan urunler:

1. Akıllı Telefon
   GTiP: 8517.12.00
   Gumruk Vergisi: %0
   KDV: %20



Thought: GTiP kodu ve vergi oranları bulundu. Şimdi CIF değerini hesaplayalım: 50 * 800 * 34
Action: calculator: 50 * 800 * 34


-- running calculator 50 * 800 * 34

[CALC] Islem: 50 * 800 * 34
Observation: 1,360,000.00

Thought: CIF değeri hesaplandı. Şimdi gümrük vergisi ve KDV'yi hesaplayalım: 1,360,000, 0, 20
Action: calculate_customs_duty: 1360000, 0, 20


-- running calculate_customs_duty 1360000, 0, 20

[TAX CALC] CIF: 1360000.0 TL, Gumruk: %0.0, KDV: %20.0
Observation: CIF: 1,360,000.00 TL, Gumruk: 0.00 TL, KDV: 272,000.00 T

In [ ]:
query("1000 spor ayakkabi ithal edecegim, 100 kg. Deniz mi hava mi daha ekonomik?")


QUESTION: 1000 spor ayakkabi ithal edecegim, 100 kg. Deniz mi hava mi daha ekonomik?

Thought: First, find the GTiP code for spor ayakkabi
Action: search_gtip_database: spor ayakkabi


-- running search_gtip_database spor ayakkabi

[GTiP SEARCH] Aranan: 'spor ayakkabi'
Observation: 'spor ayakkabi' icin bulunan urunler:

1. Spor Ayakkabı (Sneaker)
   GTiP: 6404.11.00
   Gumruk Vergisi: %8
   KDV: %20



Thought: Now that we have the GTiP code and tax rates, let's calculate the CIF value for 1000 spor ayakkabi. However, we need the unit price and the exchange rate. Since the unit price and exchange rate are not provided, let's assume the unit price is given as x USD and the exchange rate is y. We will express our calculations in terms of x and y.

Action: calculator: 1000 * x * y


-- running calculator 1000 * x * y

[CALC] Islem: 1000 * x * y
Observation: Hata: Gecersiz karakter.

Thought: It seems like we need a specific unit price to proceed. Let's assume the unit price is 50 USD per

In [ ]:
query("İthalat yaparken hangi belgeler gereklidir?")


QUESTION: İthalat yaparken hangi belgeler gereklidir?

Thought: İthalat yaparken gerekli belgeleri bulmak için gümrük mevzuatını araştırmam gerekiyor.
Action: search_knowledge_base: ithalat belgeleri


-- running search_knowledge_base ithalat belgeleri

[RAG SEARCH] Mevzuat aramasi: 'ithalat belgeleri'
Observation: Mevzuat:
Madde 67: Kıymet Beyanında Noksanlık
Kıymeti üzerinden ithalat vergilerine tabi eşyanın beyan edilen kıymeti
belirlenen kıymete göre noksan bulunduğu takdirde bu noksanlığa ait ithalat
vergilerinden başka vergi farkının üç katı para cezası alınır. Satış birimine
göre miktar itibarıyla yüzde 5'i geçmeyen bir fark ile maddi hesap hatasından
doğan noksan kıymet beyanlarında bu farklara ait ithalat vergilerinden başka
vergi farkının yarısı tutarında para cezası alınır.

Madde 58: Gümrük Yükümlülüğünün Doğması
İthalatta gümrük yükümlülüğü ithalat vergilerine tabi eşyanın serbest
dolaşıma girişi veya ithalat vergilerinden kısmi muafiyet suretiyle geçici
ithali için veril

In [ ]:
query("500 kg yuk icin deniz ve hava kargo maliyetlerini karsilastir.")


QUESTION: 500 kg yuk icin deniz ve hava kargo maliyetlerini karsilastir.

Thought: First, calculate the shipping cost for sea cargo
Action: calculate_shipping_cost: 500, deniz


-- running calculate_shipping_cost 500, deniz

[SHIPPING] 500.0 kg, Yontem: deniz
Observation: DENIZ: 4,000.00 TL

Thought: Now, calculate the shipping cost for air cargo
Action: calculate_shipping_cost: 500, hava 


-- running calculate_shipping_cost 500, hava 

[SHIPPING] 500.0 kg, Yontem: hava
Observation: HAVA: 22,500.00 TL

Thought: Compare the costs of sea and air cargo
Action: calculator: 4000, 22500
Thought: Since we just need to compare, no actual calculation is needed, we can directly observe the difference.


-- running calculator 4000, 22500

[CALC] Islem: 4000, 22500
Observation: Hata: Gecersiz karakter.

Thought: The calculator action was unnecessary, we can directly compare the costs.
Thought: Sea cargo is 4000 TL and air cargo is 22500 TL, so sea cargo is cheaper.
Answer: Deniz kargo maliyeti 4

In [ ]:
query("20000 TL CIF degeri olan bir urun icin gumruk vergisi %5, KDV %20 ise toplam ne kadar vergi oderim?")


QUESTION: 20000 TL CIF degeri olan bir urun icin gumruk vergisi %5, KDV %20 ise toplam ne kadar vergi oderim?

Thought: First, calculate customs duty
Action: calculator: 20000 * 0.05


-- running calculator 20000 * 0.05

[CALC] Islem: 20000 * 0.05
Observation: 1,000.00

Thought: Now, calculate VAT (KDV) on the total value including customs duty
Action: calculator: (20000 + 1000) * 0.20


-- running calculator (20000 + 1000) * 0.20

[CALC] Islem: (20000 + 1000) * 0.20
Observation: 4,200.00

Thought: Now, calculate the total amount including CIF value, customs duty, and VAT
Action: calculator: 20000 + 1000 + 4200


-- running calculator 20000 + 1000 + 4200

[CALC] Islem: 20000 + 1000 + 4200
Observation: 25,200.00

Thought: I have all the necessary information to calculate the total amount
Action: None needed, I can provide the answer directly
Answer: Toplam vergi ve CIF dahil 25,200.00 TL olur.

FINAL ANSWER:
Toplam vergi ve CIF dahil 25,200.00 TL olur.


In [ ]:
query("Antrepoda mal bekletmenin süre sınırı var mı?")


QUESTION: Antrepoda mal bekletmenin süre sınırı var mı?

Thought: Antrepoda mal bekletmenin süre sınırı hakkında bilgi edinmek gerekiyor.
Action: search_knowledge_base: Antrepoda mal bekletme süresi


-- running search_knowledge_base Antrepoda mal bekletme süresi

[RAG SEARCH] Mevzuat aramasi: 'Antrepoda mal bekletme süresi'
Observation: Mevzuat:
Madde 18: Antrepoda Bekleme Süreleri
Antrepoda bekletilme için minimum veya maksimum süre belirlenmemiştir. Gümrük
ve evrak işlemleri tamamlanana kadar mallar antrepoda bekletilebilir.
Malların antrepoda tutulması için belirlenmiş bir süre sınırı yoktur.

Madde 17: Antrepo Nedir
Firmalar, ithalat ve ihracat işlemlerini gerçekleştirirken belirli gümrük ve
evrak işlemleri yapmak zorundadır. Bu süreçte ilgili malların bir depoda
bekletilmesi ihtiyacı doğar. Gümrüklere yakın şekilde konumlandırılan bu
depolara antrepo adı verilir. Antrepoda bekleyen mallar henüz vergisi
ödenmemiş ticari mallar olarak nitelendirilir.

Madde 64: Gümrük Vergilerinin

In [ ]:
query("Gümrük beyannamesi nedir ve hangi belgeleri içerir?")


QUESTION: Gümrük beyannamesi nedir ve hangi belgeleri içerir?

Thought: Gümrük beyannamesi tanımını ve içeriğini arayalım
Action: search_knowledge_base: gümrük beyannamesi


-- running search_knowledge_base gümrük beyannamesi

[RAG SEARCH] Mevzuat aramasi: 'gümrük beyannamesi'
Observation: Mevzuat:
Madde 60: Gümrük Gözetiminden Çıkarma
Gümrük gözetimi altındaki ithalat vergilerine tabi eşyanın kanuna aykırı
olarak gümrük gözetimi dışına çıkarılması halinde gümrük yükümlülüğü doğar.
Gümrük yükümlülüğü eşyanın gümrük gözetiminden çıkarıldığı tarihte başlar.

Madde 43: Gümrük Rejimine Tabi Tutulma
Bir gümrük rejimine tabi tutulmak istenen eşya, bu rejime uygun şekilde
yetkili gümrük idaresine beyan edilir. İhracat, hariçte işleme, transit veya
antrepo rejimi için beyan edilen serbest dolaşımda bulunan eşya, gümrüğe
verilen beyannamenin tescilinden itibaren Türkiye Gümrük Bölgesinden
çıkıncaya veya imha edilinceye ya da gümrük beyannamesi iptal edilinceye
kadar gümrük gözetimi altında kal

In [ ]:
query("Türkiye'ye 50 iPhone (800 USD/adet), 30 iPad (1200 USD/adet) ithal edeceğim. Döviz kuru 34 TL, toplam ağırlık 10 kg, hava kargo. TOPLAM maliyeti hesapla.")


QUESTION: Türkiye'ye 50 iPhone (800 USD/adet), 30 iPad (1200 USD/adet) ithal edeceğim. Döviz kuru 34 TL, toplam ağırlık 10 kg, hava kargo. TOPLAM maliyeti hesapla.

Thought: İlk olarak iPhone ve iPad'in GTiP kodlarını bulalım.
Action: search_gtip_database: iPhone


-- running search_gtip_database iPhone

[GTiP SEARCH] Aranan: 'iPhone'
Observation: 'iPhone' icin sonuc bulunamadi.

Thought: GTiP kodu bulunamadı, daha spesifik arama yapalım.
Action: search_gtip_database: Apple iPhone

-- running search_gtip_database Apple iPhone

[GTiP SEARCH] Aranan: 'Apple iPhone'
Observation: 'Apple iPhone' icin sonuc bulunamadi.

Thought: GTiP kodu bulunamadı, genel olarak cep telefonlarının GTiP kodunu arayalım.
Action: search_gtip_database: Cep telefonu

-- running search_gtip_database Cep telefonu

[GTiP SEARCH] Aranan: 'Cep telefonu'
Observation: 'Cep telefonu' icin sonuc bulunamadi.

Thought: GTiP kodu bulunamadı, customs law'a göre cep telefonlarının GTiP kodu 8517.12.00'dir. Şimdi iPad'in GTiP

In [ ]:
query("""
Şu senaryoyu hesapla:

Şirketimiz 3 farklı ürün ithal edecek:
A) 80 LED Televizyon - 2000 USD/adet
B) 150 Akıllı Saat - 300 USD/adet
C) 50 VR Gözlüğü - 400 USD/adet

Döviz kuru: 34 TL
Toplam ağırlık: 250 kg
Taşıma: Deniz yolu

Şunları hesapla:
1. Her ürünün CIF değeri
2. Her ürünün gümrük vergisi
3. Her ürünün KDV'si
4. Navlun maliyeti
5. TOPLAM FİNAL MALİYET
""")


QUESTION: 
Şu senaryoyu hesapla:

Şirketimiz 3 farklı ürün ithal edecek:
A) 80 LED Televizyon - 2000 USD/adet
B) 150 Akıllı Saat - 300 USD/adet
C) 50 VR Gözlüğü - 400 USD/adet

Döviz kuru: 34 TL
Toplam ağırlık: 250 kg
Taşıma: Deniz yolu

Şunları hesapla:
1. Her ürünün CIF değeri
2. Her ürünün gümrük vergisi
3. Her ürünün KDV'si
4. Navlun maliyeti
5. TOPLAM FİNAL MALİYET


Thought: Önce her ürünün GTİP kodunu bulalım
Action: search_gtip_database: LED Televizyon


-- running search_gtip_database LED Televizyon

[GTiP SEARCH] Aranan: 'LED Televizyon'
Observation: 'LED Televizyon' icin bulunan urunler:

1. LED Televizyon
   GTiP: 8528.72.00
   Gumruk Vergisi: %4.9
   KDV: %20



Thought: LED Televizyon için GTİP kodu ve vergiler bulundu. Şimdi Akıllı Saat için GTİP kodunu bulalım
Action: search_gtip_database: Akıllı Saat

-- running search_gtip_database Akıllı Saat

[GTiP SEARCH] Aranan: 'Akıllı Saat'
Observation: 'Akıllı Saat' icin bulunan urunler:

1. Akıllı Saat (Smartwatch)
   GTiP: 9

In [ ]:
query("""Şirketim 30 adet No-Frost buzdolabı 500L ithal edecek. Danışmanlık için sana ihtiyacım var:ÜRÜN BİLGİLERİ:
- Miktar: 30 adet
- Birim fiyat: 200 USD
- Döviz kuru: 34 TL
- Ürün: Buzdolabı (No-Frost, 500L)
- Ağırlık: 100 kg
- Taşıma: Deniz yolu

SORULARIM:

1. MALİYET HESAPLAMA:
   - CIF değeri nedir?
   - Gümrük vergisi ne kadar?
   - KDV ne kadar?
   - Navlun maliyeti ne kadar?
   - TOPLAM ithalat maliyeti ne kadar?

2. İTHALAT BELGELERİ:
   - Bu ithalat için hangi belgeler gereklidir?
   - Listele ve kısaca açıkla.
?""")


QUESTION: Şirketim 30 adet No-Frost buzdolabı 500L ithal edecek. Danışmanlık için sana ihtiyacım var:ÜRÜN BİLGİLERİ:
- Miktar: 30 adet
- Birim fiyat: 200 USD
- Döviz kuru: 34 TL
- Ürün: Buzdolabı (No-Frost, 500L)
- Ağırlık: 100 kg
- Taşıma: Deniz yolu

SORULARIM:

1. MALİYET HESAPLAMA:
   - CIF değeri nedir?
   - Gümrük vergisi ne kadar?
   - KDV ne kadar?
   - Navlun maliyeti ne kadar?
   - TOPLAM ithalat maliyeti ne kadar?

2. İTHALAT BELGELERİ:
   - Bu ithalat için hangi belgeler gereklidir?
   - Listele ve kısaca açıkla.
?

Thought: Önce CIF değerini hesaplayalım. 
Action: calculator: 30 * 200 * 34


-- running calculator 30 * 200 * 34

[CALC] Islem: 30 * 200 * 34
Observation: 204,000.00

Thought: CIF değeri 204,000.00 TL. Şimdi GTİP kodunu bulalım.
Action: search_gtip_database: No-Frost buzdolabı 500L


-- running search_gtip_database No-Frost buzdolabı 500L

[GTiP SEARCH] Aranan: 'No-Frost buzdolabı 500L'
Observation: 'No-Frost buzdolabı 500L' icin sonuc bulunamadi.



RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kdjpjgkkft9vsfj2q510f2b2` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99542, Requested 1332. Please try again in 12m35.136s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [12]:
from groq import Groq

client = Groq(api_key=userdata.get('GROQ_API_KEY'))

def weak_base_model_query(question):
    messages = [{"role": "user", "content": question}]
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# TEST SORUSU
test_question = """İthalat yaparken hangi belgeler gereklidir?
"""

base_result = weak_base_model_query(test_question)
print(base_result)
print("\n")


İthalat yaparken aşağıdaki belgeler gereklidir:

1. **İthalat İzin Belgesi**: İthalat yapmadan önce, ithalatçı firma tarafından ithalat izin belgesi alınması gereklidir. Bu belge, ithalatın hangi ürünler için yapıldığını ve hangi miktarlarda yapıldığını gösterir.
2. **Fatura**: İthalatçı firma tarafından düzenlenen fatura, ithal edilen ürünlerin fiyatını ve miktarını gösterir.
3. **İthalat Belgesi (İB)**: İthalatçı firma tarafından düzenlenen ithalat belgesi, ithal edilen ürünlerin tanımı, miktarı, fiyatı ve diğer detaylarını gösterir.
4. **İthalat İzin Belgesi (İİB)**: İthalatçı firma tarafından düzenlenen ithalat izin belgesi, ithal edilen ürünlerin ithalatını onaylayan belgedir.
5. **Gümrük Beyannamesi**: İthalatçı firma tarafından düzenlenen gümrük beyannamesi, ithal edilen ürünlerin gümrük vergilerini gösterir.
6. **İthalatçı Firması Kaydı**: İthalatçı firma, gümrük müdürlüğüne kaydolmalıdır.
7. **İthalatçı Firması Yetkilisi Belgesi**: İthalatçı firma yetkilisi, gümrük müdürlüğüne

In [13]:
from groq import Groq

client = Groq(api_key=userdata.get('GROQ_API_KEY'))

def weak_base_model_query(question):
    messages = [{"role": "user", "content": question}]
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# TEST SORUSU
test_question = """50 akilli telefon ithal etmek istiyorum. Birim fiyat 800 USD, kur 34 TL, toplam agirlik 10 kg, hava kargo. Toplam maliyet ne kadar?

"""

base_result = weak_base_model_query(test_question)
print(base_result)
print("\n")


50 akilli telefon ithal etmek istediğinizde, toplam maliyeti hesaplamak için aşağıdaki adımları takip edebilirsiniz:

1. Birim fiyat: 800 USD
2. Kur: 34 TL/USD
3. Toplam akilli telefon sayısı: 50
4. Toplam birim fiyat: 800 USD x 50 = 40.000 USD
5. Toplam TL maliyeti: 40.000 USD x 34 TL/USD = 1.360.000 TL
6. Toplam ağırlık: 10 kg
7. Hava kargo ücreti: Bu ücret genellikle ağırlık ve boyutlara göre değişir. Ancak ortalama bir hava kargo ücreti için, 10 kg için 150-200 TL arasında bir ücret alabiliriz. Bu örnek için 175 TL olarak kabul edelim.
8. Toplam maliyet: 1.360.000 TL + 175 TL = 1.360.175 TL

Bu hesaplamalar sonucunda, 50 akilli telefon ithal etmek için toplam maliyetiniz 1.360.175 TL olacaktır.


